<a href="https://colab.research.google.com/github/guthi1/deep-learning/blob/main/Assignment_1/solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IFT6135-A2022
# Assignment 1: Practical

You must fill in your answers to various questions in this notebook, following which you must export this notebook to a Python file named `solution.py` and submit it on Gradescope.

Only edit the functions specified in the PDF (and wherever marked – `# WRITE CODE HERE`). Do not change definitions or edit the rest of the template, else the autograder will not work.

**Make sure you request a GPU runtime (especially for Question 3)!**

In [1]:
# DO NOT MODIFY!
import matplotlib.pyplot as plt
import numpy as np
import random

# Fix random seed
random.seed(0)
np.random.seed(0)

## Question 1: Implementing MLPs with NumPy (30 points)

In [100]:
class NN(object):
  """
    Implements an MLP.
  """

  def __init__(self,
               dims=(784, 128, 64, 10), # h_0, h_1, h_2, h_3
               activation="relu",       # Activation function
               epsilon=1e-6,            # Correction factor
               lr=0.01,                 # Learning rate
               seed=0                   # Random seed
              ):
    """
      Constructor of the NN class.

      dims: list or tuple or np.array, default (784, 128, 64, 10)
        Values of h_0 (no. of features), h_1 (hidden dim. 1), h_2 (hidden dim. 2), h_3 (no. of output classes).
      activation: string, default "relu"
        Activation function to use.
      epsilon: float or double, default 1e-6
        Correction factor to clip probabilities.
      lr: float or double, default 0.01
        Learning rate for weight updates.
      seed: int, default 0
        Random seed.
    """
    super(NN, self).__init__()

    self.dims = dims
    self.n_hidden = len(dims) - 2
    self.activation_str = activation
    self.epsilon = epsilon
    self.lr = lr
    self.seed = seed
    self.debug = False

  def initialize_weights(self):
    """
      Results: Initializes the weights of the MLP from uniform(-1/sqrt(h_0), 1/sqrt(h_0)) and the biases to zeros.
    """
    if self.seed is not None:
      np.random.seed(self.seed)

    self.weights = {}
    # self.weights is a dictionary with keys W1, b1, W2, b2, ..., Wm, Bm where m - 1 is the number of hidden layers
    # The keys W1, W2, ..., Wm correspond to weights while b1, b2, ..., bm correspond to biases
    for layer_n in range(1, self.n_hidden + 2):
      """ h_0 = nb. feature input data """
      h_0 = self.dims[0]

      """ Number of weight of a layer """
      n_weights = self.dims[layer_n-1] * (self.dims[layer_n])

      """ Init the weight """
      weights = np.random.uniform(-1/np.sqrt(h_0), 1/np.sqrt(h_0), size=n_weights)

      """ Reshape in the good size
      " | W_00  W_01 .. |
      " | W_10  W_11 .. |
      " |  ..    ..  .. | 
      """
      weights = np.reshape(weights, (self.dims[layer_n-1], self.dims[layer_n]))

      """ Weights : matrix of shape(n_neurone{layer_i}, n_neuro{layer_i-1}) """
      self.weights[f"W{layer_n}"] = weights
      self.weights[f"b{layer_n}"] = np.zeros((1, self.dims[layer_n]))

  def relu(self, x, grad=False):
    """
      x: np.array
        Inputs to calculate ReLU(x) for. x may contain a batch of inputs!
      grad: bool, default False
        If True, return the gradient of the activation with respect to the inputs to the function.

      Outputs: Implements the ReLU activation function or its gradient.
    """
    if grad:
      x[x <= 0] = 0
      x[x > 0] = 1
      return x
    return np.maximum(0, x)

  def sigmoid(self, x, grad=False):
    """
      x: np.array
        Inputs to calculate sigmoid(x) for. x may contain a batch of inputs!
      grad: bool, default False
        If True, return the gradient of the activation with respect to the inputs to the function.

      Outputs: Implements the Sigmoid activation function or its gradient.
    """
    def sigmoid(x):
        return 1/(1+np.exp(-x))

    if grad:
      return sigmoid(x)*(1-sigmoid(x))
    return sigmoid(x)

  def tanh(self, x, grad=False):
    """
      x: np.array
        Inputs to calculate tanh(x) for. x may contain a batch of inputs!
      grad: bool, default False
        If True, return the gradient of the activation with respect to the inputs to the function.

      Outputs: Implements the tanh activation function or its gradient.
    """
    def tanh(x):
        return (np.exp(x)-np.exp(-x))/(np.exp(x)+np.exp(-x))

    if grad:
      return 1-(np.power(tanh(x), 2))
    return tanh(x)

  def activation(self, x, grad=False):
    """
      x: np.array
        Inputs to calculate activation(x) for. x may contain a batch of inputs!
      grad: bool, default False
        If True, return the gradient of the activation with respect to the inputs to the function.

      Outputs: Returns the value of the activation or the gradient.
    """
    if self.activation_str == "relu":
      return self.relu(x, grad=grad) 
    elif self.activation_str == "sigmoid":
      return self.sigmoid(x, grad=grad)
    elif self.activation_str == "tanh":
      return self.tanh(x, grad=grad)
    else:
      raise Exception("Invalid activation")
    return 0

  def softmax(self, x):
    """
      x: np.array
        Inputs to calculate softmax over. x may contain a batch of inputs!

      Outputs: Implements the softmax function, returns the array containing softmax(x).
    """
    axis = 1
    max = np.max(x, axis=axis)
    exp_x = np.exp(x - np.expand_dims(max, axis))
    sum = np.sum(exp_x, axis=axis)
    return exp_x / np.expand_dims(sum, axis)

  def forward(self, x):
    """
      x: np.array
        Inputs to the MLP. Note that x may contain multiple input examples, not just one example.

      Outputs: Implements the forward pass, returns cache as described below.
    """
    cache = {"Z0": x}
    # cache is a dictionary with keys Z0, A1, Z1, ..., Am, Zm where m - 1 is the number of hidden layers
    # Z0 just contains the inputs x to the network
    # Ai corresponds to the preactivation at layer i, Zi corresponds to the activation at layer i
    if self.debug:print("\nFoward ...")
    z = x

    for layer_n in range(1, self.n_hidden + 2):

        # Get the weiths
        weights = self.weights[f"W{layer_n}"]
        bias = self.weights[f"b{layer_n}"]

        # Calcul       
        if self.debug:print(f"layer_{layer_n}")
        if self.debug:print(f"    z_{z.shape} dot w_{weights.shape} + b_{bias.shape}")
        # Do calcul
        a = z.dot(weights) + bias
        
        if layer_n == self.n_hidden+1:
            z = self.softmax(a)
        else:
            z = self.activation(a)
        if self.debug:print(f"    = z_{z.shape}")
    
        # Save value
        cache[f"A{layer_n}"] = a
        cache[f"Z{layer_n}"] = z
    
    return cache

  def loss(self, prediction, labels):
    """
      prediction: np.array
        Predicted probabilities for each class for inputs. May contain multiple examples (a batch)!
      labels: np.array
        True labels corresponding to the inputs (assume they are one-hot encoded). May contain multiple examples (a batch)!

      Outputs: Returns the crossentropy loss (take the mean over number of inputs).
    """
    prediction[np.where(prediction < self.epsilon)] = self.epsilon
    prediction[np.where(prediction > 1 - self.epsilon)] = 1 - self.epsilon
    return (- np.sum(labels * np.log(prediction))) / len(prediction)

  def backward(self, cache, labels):
    """
      cache: np.array
        Results of the backward pass. This may be for multiple examples (a batch).
      labels: np.array
        True labels corresponding to the inputs in cache. May contain multiple examples (a batch)!

      Outputs: Implements the backward pass, returns grads as described below.
    """
    output = cache[f"Z{self.n_hidden + 1}"]
    grads = {}

    # grads is a dictionary with keys dAm, dWm, dbm, dZ(m-1), dA(m-1), ..., dW1, db1
    # Remember to account for the number of input examples!

    """
    We need to find: dC/dw et dC/db. For that, let introduce intermediate quantity
    delta_l: [] vector of errors associated with neuron in layer l
        -> delta_l = dC/ da
        -> Come from activation(a_l + little_delta)
        -> delta_l = dC/dz_l = activation'(a_l)
    """
    if self.debug:print("\nBackward ...")
    
    # Last layer
    """ delta_L = nabla Hadamard_product act_func_derivative(w_L * h_L-1 + b) """
    if self.debug:print(f"last layer ... layer_{self.n_hidden + 1} ----------")
    if self.debug:print(f"   -> delta")
    if self.debug:print(f"      delta = pred_{output.shape} - labels_{labels.shape}")
    delta = (output - labels) # softmax cross_entropy backward
    bias_grad = np.mean(delta, axis=0)
    if self.debug:print(f"      delta_{delta.shape}")
    # Save value
    grads[f"db{self.n_hidden + 1}"] = bias_grad
    grads[f"dA{self.n_hidden + 1}"] = delta

    """ dC/dw_L = h_L-1 * delta_L """
    if self.debug:print(f"   -> Update")
    z_in = cache[f"Z{self.n_hidden}"]
    z_in = np.mean(z_in, axis=0)# TODO: FIX
    if self.debug:print(f"      partial_derivatire = z_in_{z_in.shape} * delta_{delta.shape}")
    partial_derivatire = np.multiply.outer(z_in, delta)
    partial_derivatire = np.mean(partial_derivatire, axis=1) # TODO: FIX
    if self.debug:print(f"      partial_derivatire_{partial_derivatire.shape}")
    # Save value
    grads[f"dW{self.n_hidden + 1}"] = partial_derivatire

    delta = np.mean(delta, axis=0) # TODO: REMOVE


    # From here, we need to apply the chain rule -  Loop backward
    for layer_n in range(self.n_hidden, 0, -1):
        if self.debug:print(f"Looping backward ... layer_{layer_n} ----------")
        """ delta_l = ((w_l+1).T * delta_l+1) element_wise_mul act_func_derivative(w_l * z_l-1 + b) """
        if self.debug:print(f"   -> delta")
        a = cache[f"A{layer_n}"]
        activation_back = self.activation(a, grad=True)
        activation_back = np.mean(activation_back, axis=0) # TODO: REMOVE
        # Find error (delta) for the current layer
        weights_next = self.weights[f"W{layer_n+1}"]
        if self.debug:print(f"      delta_pre_act = weight_{weights_next.T.shape} * delta_{delta.shape}")
        delta_pre_act = np.matmul(weights_next, delta)
        grads[f"dA{layer_n}"] = delta_pre_act
        # error * activation_back
        if self.debug:print(f"      delta_ = delta_pre_act_{delta_pre_act.shape} * activation_back_{activation_back.shape}")
        delta = delta_pre_act * activation_back
        if self.debug:print(f"      delta_{delta.shape}")

        grads[f"db{layer_n}"] = delta

        """ dC/dw_l = z_l-1 * delta_l """
        if self.debug:print(f"   -> Update")
        z_in = cache[f"Z{layer_n-1}"]
        z_in = np.mean(z_in, axis=0)
        if self.debug:print(f"      partial_derivatire = z_in_{z_in.shape} * delta_{delta.shape}")
        partial_derivatire = np.multiply.outer(z_in, delta)
        if self.debug:print(f"      partial_derivatire_{partial_derivatire.shape}")

        grads[f"dW{layer_n}"] = partial_derivatire

    return grads

  def update(self, grads):
    """
      grads: np.dictionary
        Gradients obtained from the backward pass.

      Results: Updates the network's weights and biases.
    """
    if self.debug:print(f"\nUpdate ... ~~~~~~~~~~~~~~~~")
    for layer in range(1, self.n_hidden + 2):
        # WRITE CODE HERE
        if self.debug:print(f"layer_{layer}")

        grad = grads[f"dW{layer}"]
        weights = self.weights[f"W{layer}"]
        if self.debug:print(f"   weights_{weights.shape} - lr * grad_{grad.shape}")
        weights = weights - self.lr * grad
        self.weights[f"W{layer}"] = weights

        bias_up = grads[f"db{layer}"]
        bias = self.weights[f"b{layer}"]
        if self.debug:print(f"   weights_{bias.shape} - lr * bias_up_{bias_up.shape}")
        bias = bias - self.lr * bias_up
        self.weights[f"b{layer}"] = bias

In [101]:
x = np.random.rand(5, 784)
labels = np.zeros((5, 10))
labels[:,:1] = 1
print(labels)
print(f"Input size: {x.shape}")

nn = NN()
nn.debug = True
nn.initialize_weights()
res = nn.forward(x)
output = res["Z3"]
print(output)
training_loop = 1
loss = nn.loss(output, labels)
print(f"Loss: {loss}")

for _ in range(training_loop):
    res = nn.forward(x)
    output = res["Z3"]
    loss = nn.loss(output, labels)
    print(f"Loss: {loss}")

    grads = nn.backward(res, labels)
    nn.update(grads)

res = nn.forward(x)
output = res["Z3"]
print(output)

np.argmax(output, axis=1)

[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
Input size: (5, 784)

Foward ...
layer_1
    z_(5, 784) dot w_(784, 128) + b_(1, 128)
    = z_(5, 128)
layer_2
    z_(5, 128) dot w_(128, 64) + b_(1, 64)
    = z_(5, 64)
layer_3
    z_(5, 64) dot w_(64, 10) + b_(1, 10)
    = z_(5, 10)
[[0.09931704 0.09984055 0.09966514 0.10010934 0.10070109 0.09965174
  0.10011623 0.09913298 0.10056654 0.10089934]
 [0.09861982 0.09952156 0.09981243 0.10005793 0.10038148 0.1002567
  0.0999746  0.09990091 0.10073391 0.10074066]
 [0.0986791  0.09993813 0.09993374 0.10019995 0.10075545 0.09977775
  0.0999143  0.09976137 0.10041154 0.10062866]
 [0.09824672 0.09960783 0.10002933 0.09993164 0.10039489 0.10027982
  0.10029415 0.09969323 0.10058814 0.10093425]
 [0.09928766 0.09987312 0.09996769 0.09985034 0.10001318 0.10003813
  0.09966909 0.09961099 0.10088057 0.10080925]]
Loss: 2.3143621413221735

array([9, 9, 4, 9, 8])

## Question 2: Implementing CNN layers with NumPy (20 points)
Note: You may assume that there are no biases, no input padding (valid convolution) and also that convolution here refers to cross-correlation, i.e., no kernel flipping when convolving the inputs.

In [ ]:
class Convolution2dLayer(object):
  """
    Implements a 2D convolution layer.
  """

  def __init__(self, filter_size=3, stride=1, n_units=64, seed=0):
    """
      Constructor of the Convolution2dLayer class.

      Note: We assume that the input images have only a single channel.

      filter_size: int, default 3
        Filter size to use for convolution. We assume equal height and width.
      stride: int, default 1
        Stride for convolution.
      n_units: int, default 64
        Number of output channels, i.e., number of filters in the layer.
      seed: int, default 0
        Random seed.
    """
    super(Convolution2dLayer, self).__init__()

    self.filter_size = filter_size
    self.stride = stride
    self.n_units = n_units
    self.seed = seed

  def initialize_weights(self):     
    """
      Results: Initializes the weights of the CNN from uniform(0, 1).
    """   
    if self.seed is not None:
      np.random.seed(self.seed)

    # self.weights is an np.array of shape (n_units, filter_size, filter_size)
    # We do not consider biases in this convolution layer implementation
    # WRITE CODE HERE
    self.weights = None

  def forward(self, x):
    """
      x: np.array
        Inputs to convolve. This may contain multiple input examples, not just one.
        Note: We assume that the input images have only a single channel, e.g. (5, 1, 32, 32) where 5 is the number of
        images, 1 channel, 32x32 image size.

      Outputs: Inputs and the result of the convolution operation on the inputs stored in cache.

      Note: You need not flip the kernel! You may just implement cross-correlation.
    """
    cache = {}

    # cache is a dictionary where cache["x"] stores the inputs and cache["out"] stores the outputs of the layer
    # WRITE CODE HERE
    cache["x"] = x
    cache["out"] = None
    pass
    return cache

  def backward(self, cache, grad_output):
    """
      cache: dictionary
        Contains the inputs and the result of the convolution operation applied on them.
      grad_output: np.array
        Gradient of the loss with respect to the outputs of the convolution layer.

      Outputs: Gradient of the loss with respect to the parameters of the convolution layer.
    """
    # grads is an np.array containing the gradient of the loss with respect to the parameters in the convolution layer
    # Remember to account for the number of input examples!
    # WRITE CODE HERE
    grads = None
    pass
    return grads


class MaxPooling2dLayer(object):
  """
    Implements a 2D max-pooling layer.
  """

  def __init__(self, filter_size=2):
    """
      Constructor of the MaxPooling2dLayer class.

      filter_size: int, default 2
        Filter size to use for max-pooling. We assume equal height and width, and stride = height = width.
    """
    super(MaxPooling2dLayer, self).__init__()
  
    self.filter_size = filter_size
    self.stride = filter_size

  def forward(self, x):
    """
      x: np.array
        Inputs to compute max-pooling for. This may contain multiple input examples, not just one.
        Note: The input dimensions to max-pooling are the output dimensions of the convolution!

      Outputs: Inputs and the result of the max-pooling operation on the inputs stored in cache.
    """
    cache = {}

    # cache is a dictionary where cache["x"] stores the inputs and cache["out"] stores the outputs of the layer
    # WRITE CODE HERE
    cache["x"] = x
    cache["out"] = None
    return cache

  def backward(self, cache, grad_output):
    """
      cache: dictionary
        Contains the inputs and the result of the max-pooling operation applied on them.
      grad_output: np.array
        Gradient of the loss with respect to the outputs of the max-pooling layer.

      Outputs: Gradient of the loss with respect to the inputs to the max-pooling layer.
    """
    grads = None # WRITE CODE HERE (initialize grads correctly)

    # grads is an np.array containing the gradient of the loss with respect to the inputs to the max-pooling layer
    # Remember to account for the number of input examples!
    # WRITE CODE HERE
    pass
    return grads

## Question 3: Implementing a CNN and comparison with MLPs using PyTorch (50 points)

In [ ]:
# DO NOT MODIFY!
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
device = "cuda" if torch.cuda.is_available() else "cpu"

# Fix random seed
torch.manual_seed(0)
torch.cuda.manual_seed_all(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [ ]:
class ResidualBlock(nn.Module):
  """This class implements the Residual Block used in ResNet-18."""

  def __init__(self, in_channels, channels, conv_stride=1, activation_str="relu", initialization="xavier_normal"):
    """
      Constructor for the ResidualBlock class.

      in_channels: int
        Number of channels in the input to the block.
      channels: int
        Number of output channels for the block, i.e., number of filters.
      conv_stride: int, default 1
        Stride of the first convolution layer and downsampling convolution (if required).
      activation_str: string, default "relu"
        Activation function to use.
      initialization: string, default "xavier_normal"
        Initialization for convolution layer weights.
    """
    super(ResidualBlock, self).__init__()

    self.in_channels = in_channels
    self.channels = channels
    self.conv_stride = conv_stride
    self.activation_str = activation_str
    self.initialization = initialization

    # Define these members by replacing `None` with the correct definitions
    self.conv1 = None # WRITE CODE HERE
    self.bn1 = None   # WRITE CODE HERE
    self.conv2 = None # WRITE CODE HERE
    self.bn2 = None   # WRITE CODE HERE

    self.residual_connection = self.residual(in_channels, channels, conv_stride)

    # Initialize weights for conv1 and conv2
    if initialization == "xavier_normal":
      # WRITE CODE HERE
      pass
    elif initialization == "xavier_uniform":
      # WRITE CODE HERE
      pass
    elif initialization == "kaiming_normal":
      # WRITE CODE HERE
      pass
    else:
      raise Exception("Invalid initialization")

  def activation(self, input):
    """
      input: Tensor
        Input on which the activation is applied.

      Output: Result of activation function applied on input.
        E.g. if self.activation_str is "relu", return relu(input).
    """
    if self.activation_str == "relu":
      # WRITE CODE HERE
      pass
    elif self.activation_str == "tanh":
      # WRITE CODE HERE
      pass
    else:
      raise Exception("Invalid activation")
    return 0

  def residual(self, in_channels, channels, conv_stride=1):
    """
      in_channels: int
        Number of input channels in the input to the block.
      channels: int
        Number of output channels for the block, i.e., number of filters.
      conv_stride: int, default 1
        Stride to use for downsampling 1x1 convolution.

      Output: Returns an nn.Sequential object which computes the identity function of the input if stride is 1
              and the number of input channels equals the number of output channels. Otherwise, it returns an
              nn.Sequential object that downsamples its input using a 1x1-conv of the stride specified and
              followed by a BatchNorm2d.
    """
    layers = []
    if conv_stride != 1 or in_channels != channels:
      # WRITE CODE HERE
      pass
    return nn.Sequential(*layers)

  def forward(self, x):
    """
      x: Tensor
        Input to the block.

      Outputs: Returns the output of the forward pass of the block.
    """
    # WRITE CODE HERE
    pass
    return 0

In [ ]:
class ResNet18(nn.Module):
  """This class implements the ResNet-18 architecture from its components."""

  def __init__(self, activation_str="relu", initialization="xavier_normal"):
    """
      Constructor for the ResNet18 class.

      activation_str: string, default "relu"
        Activation function to use.
      initialization: string, default "xavier_normal"
        Weight initialization to use.
    """
    super(ResNet18, self).__init__()

    self.n_classes = 10
    self.activation_str = activation_str
    self.initialization = initialization

    # Define these members by replacing `None` with the correct definitions
    self.conv1 = None   # WRITE CODE HERE
    self.bn1 = None     # WRITE CODE HERE
    self.layer1 = None  # WRITE CODE HERE (use _create_layer)
    self.layer2 = None  # WRITE CODE HERE (use _create_layer)
    self.layer3 = None  # WRITE CODE HERE (use _create_layer)
    self.layer4 = None  # WRITE CODE HERE (use _create_layer)
    self.avgpool = None # WRITE CODE HERE
    self.linear = None  # WRITE CODE HERE
  
  def activation(self, input):
    """
      input: Tensor
        Input on which the activation is applied.

      Output: Result of activation function applied on input.
        E.g. if self.activation_str is "relu", return relu(input).
    """
    if self.activation_str == "relu":
      # WRITE CODE HERE
      pass
    elif self.activation_str == "tanh":
      # WRITE CODE HERE
      pass
    else:
      raise Exception("Invalid activation")
    return 0

  def _create_layer(self, in_channels, channels, conv_stride=1):
    """
      in_channels: int
        Number of input channels present in the input to the layer.
      out_channels: int
        Number of output channels for the layer, i.e., the number of filters.
      conv_stride: int, default 1
        Stride of the first convolution layer in the block and the downsampling convolution (if required).

      Outputs: Returns an nn.Sequential object giving a "layer" of the ResNet, consisting of 2 blocks each.
    """
    # Modify the following statement to return an nn.Sequential object containing 2 ResidualBlocks.
    # You must make sure that the appropriate channels and conv_stride are provided.
    return nn.Sequential() # WRITE CODE HERE

  def get_first_conv_layer_filters(self):
    """
      Outputs: Returns the filters in the first convolution layer.
    """
    return self.conv1.weight.clone().cpu().detach().numpy()

  def get_last_conv_layer_filters(self):
    """
      Outputs: Returns the filters in the last convolution layer.
    """
    return list(self.layer4.modules())[1].conv2.weight.clone().cpu().detach().numpy()

  def forward(self, x):
    """
      x: Tensor
        Input to the network.

      Outputs: Returns the output of the forward pass of the network.
    """
    # WRITE CODE HERE
    pass
    return 0

In [ ]:
def get_cifar10():  
  transform = transforms.Compose([
      transforms.ToTensor()
  ])

  train_dataset = torchvision.datasets.CIFAR10(
      root='./data', train=True, download=True, transform=transform)
  train_loader = torch.utils.data.DataLoader(
      train_dataset, batch_size=128, shuffle=True, num_workers=2)

  val_dataset = torchvision.datasets.CIFAR10(
      root='./data', train=False, download=True, transform=transform)
  val_loader = torch.utils.data.DataLoader(
      val_dataset, batch_size=128, shuffle=False, num_workers=2)
  
  return train_loader, val_loader

def train_loop(epoch, model, train_loader, criterion, optimizer):
  """
    epoch: int
      Number of the current training epoch (starting from 0).
    model: ResNet18
      The model to train, which is an instance of the ResNet18 class.
    train_loader: DataLoader
      The training dataloader.
    criterion: Module
      A Module object that evaluates the crossentropy loss.
    optimizer: Optimizer
      An Optimizer object for the Adam optimizer.

    Outputs: Returns average train_acc and train_loss for the current epoch.
  """
  train_acc = 0.
  train_loss = 0.

  # WRITE CODE HERE

  print(f"Epoch: {epoch} | Train Acc: {train_acc:.6f} | Train Loss: {train_loss:.6f}")
  return train_acc, train_loss

def valid_loop(epoch, model, val_loader, criterion):
  """
    epoch: int
      Number of the current epoch (starting from 0).
    model: ResNet18
      The model to train, which is an instance of the ResNet18 class.
    val_loader: DataLoader
      The validation dataloader.
    criterion: Module
      A Module object that evaluates the crossentropy loss.

    Outputs: Returns average val_acc and val_loss for the current epoch.
  """
  val_acc = 0.
  val_loss = 0.

  # WRITE CODE HERE

  print(f"Epoch: {epoch} | Val Acc: {val_acc:.6f}   | Val Loss: {val_loss:.6f}")
  return val_acc, val_loss

In [ ]:
activation_str = "relu"
initialization = "xavier_normal"

In [ ]:
if __name__ == "__main__":
  train_accs, train_losses, val_accs, val_losses = [], [], [], []
  n_epochs = 25

  model = ResNet18(
    activation_str=activation_str,
    initialization=initialization
  ).to(device)
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.Adam(model.parameters())

  train_loader, val_loader = get_cifar10()

  for epoch in range(n_epochs):
    # Training
    train_acc, train_loss = train_loop(epoch, model, train_loader, criterion, optimizer)
    train_accs.append(train_acc)
    train_losses.append(train_loss)

    # Validation
    val_acc, val_loss = valid_loop(epoch, model, val_loader, criterion)
    val_accs.append(val_acc)
    val_losses.append(val_loss)

### Questions 3.4, 3.5, 3.6, 3.7, 3.8
You may write your own code for these questions below. These will not be autograded and you need not submit code for these, only the report.

In [ ]:
# For Q 3.6
if __name__ == "__main__":
  vis_image = None
  for data, labels in val_loader:
    vis_image = data[12].unsqueeze(0)
    break

In [ ]:
# You can also upload `vis_image.pkl` from Piazza and use it:
# import pickle
# vis_image = pickle.load(open("vis_image.pkl", "rb")).to(device)
# plt.imshow(vis_image.squeeze().permute(1, 2, 0).cpu().detach().numpy())

In [ ]:
# import matplotlib.pyplot as plt
# plt.imshow(vis_image.squeeze().permute(1, 2, 0).cpu().detach().numpy())